# villaInventorySdk
[docs](https://thanakijwanavit.github.io/villa-inventory-sdk/)
> read and write inventory in real time

full docs here https://thanakijwanavit.github.io/villa-inventory-sdk/

## Install

`pip install villaInventorySdk`

## How to use

Uploading a large amount of data

In [ ]:
#hide
# isAws = True
# if isAws:
#   USER = None
#   PW = None
# else:
#   import pickle
#   KEY = ''
#   PW = ''
#   keypath = '/Users/nic/.villa-product-master-inventory'
#   if KEY and PW:
#     with open (keypath, 'wb') as f:
#       pickle.dump({
#           'KEY': KEY,
#           'PW': PW
#       }, f)
#   with open(keypath, 'rb') as f:
#     creden = pickle.load(f)
#   USER = creden['KEY']
#   PW = creden['PW']

## sample input

In [ ]:
from villaInventorySdk.inventory import InventorySdk
from random import randrange
import boto3, time, json
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from datetime import datetime
import pandas as pd
from nicHelper.dictUtil import printDict

In [ ]:
sampleInput =  [ 
  { 'cprcode': '0000009', 'brcode': '1000', 'ib_cf_qty': '50', 'new_ib_vs_stock_cv': '27' },
  { 'cprcode': '0000004', 'brcode': '1000', 'ib_cf_qty': '35', 'new_ib_vs_stock_cv': '33' },
  { 'cprcode': '0000003', 'brcode': '1003', 'ib_cf_qty': '36', 'new_ib_vs_stock_cv': '33' }
    ]
df = pd.DataFrame(sampleInput)
df

,cprcode,brcode,ib_cf_qty,new_ib_vs_stock_cv
0,0000009,1000,50,27
1,0000004,1000,35,33
2,0000003,1003,36,33


## Upload data

In [ ]:
#hide
inputBucketName= 'input-bucket-dev-manual'
functionName= 'update-inventory-s3-dev-manual'
invocationType = 'RequestResponse'
branch = 'dev-manual'

In [ ]:
#hide
inputBucketName= 'input-bucket-test-inventory-6b3753e2f2'
functionName= 'update-inventory-s3-test-inventory-6b3753e2f2'
invocationType = 'RequestResponse'

## init sdk

In [ ]:
%%time
USER=None
PW = None
sdk = InventorySdk(user=USER, pw=PW, branchName = branch)

CPU times: user 36.2 ms, sys: 4.05 ms, total: 40.2 ms
Wall time: 39.8 ms


## Update inventory 

In [ ]:
%%time
key = 'test'
r = sdk.uploadDf(df, key = key)
if r.status_code >= 400: raise Exception(r.json())
sdk.ingestData(key = key)

signed url is 
url : https://in
fields
 key : test
 AWSAccessKeyId : ASIAVX4Z5T
 x-amz-security-token : IQoJb3JpZ2
 policy : eyJleHBpcm
 signature : JhA/noZmLG
CPU times: user 58.8 ms, sys: 1.44 ms, total: 60.2 ms
Wall time: 574 ms


{'body': '{"cprcode":{"0":"0000009","1":"0000004","2":"0000003"},"brcode":{"0":"1000","1":"1000","2":"1003"},"ib_cf_qty":{"0":"50","1":"35","2":"36"},"new_ib_vs_stock_cv":{"0":"27","1":"33","2":"33"}}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

#### test uploading full data 

In [ ]:
#### test uploading real data 
df = pd.read_csv('sampleData/inventory.csv', index_col=0, dtype=str).reset_index(drop=True)
r = sdk.uploadDf(df, key = key)
if r.status_code >= 400: raise Exception(r.json())
sdk.ingestData(key = key)

signed url is 
url : https://in
fields
 key : test
 AWSAccessKeyId : ASIAVX4Z5T
 x-amz-security-token : IQoJb3JpZ2
 policy : eyJleHBpcm
 signature : 7846JjbAR+


{'body': '{"cprcode":{"0":"0000009","1":"0000012","2":"0000026","3":"0000028","4":"0000033"},"brcode":{"0":"1000","1":"1000","2":"1000","3":"1000","4":"1000"},"ib_cf_qty":{"0":"39","1":"39","2":"9","3":"13","4":"7"},"new_ib_vs_stock_cv":{"0":"39","1":"39","2":"9","3":"13","4":"7"}}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

## Query single product

In [ ]:
%%time
sdk.querySingleProduct2(cprcode='1234')

succesfully get url, returning pandas
CPU times: user 33.8 ms, sys: 9.14 ms, total: 42.9 ms
Wall time: 3 s


,cprcode,brcode,ib_cf_qty,new_ib_vs_stock_cv
0,1234,test,123,123


## Query Branch

In [ ]:
%%time
sdk.branchQuery(brcode='1000', cprcodes = [9])

CPU times: user 15.3 ms, sys: 1.2 ms, total: 16.5 ms
Wall time: 231 ms


,cprcode,brcode,ib_cf_qty,new_ib_vs_stock_cv
0,9,1000,50,27
1,9,1000,50,27


## Query All

In [ ]:
%%time
sdk.queryAll2()

succesfully get url, returning pandas
CPU times: user 21.4 ms, sys: 15.7 ms, total: 37.1 ms
Wall time: 420 ms


,cprcode,brcode,ib_cf_qty,new_ib_vs_stock_cv
0,1234,test,123,123
1,12345,test,345,345
2,4,1000,35,33
3,4,1000,35,33
4,9,1000,50,27
...,...,...,...,...
46978,244590,1000,0,0
46979,244591,1000,0,0
46980,244592,1000,0,0
46981,3,1003,36,33
